In [2]:
import numpy as np
import random
import tensorflow as tf  # Import TensorFlow if you're using it
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

# Load your dataset and preprocess it
# Make sure you've properly loaded the dataset and assigned it to the 'dataset' variable

cocomo=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomo81_dataset.csv",header=None)
columns_cocomo=['rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','effort']
cocomo.set_axis(columns_cocomo,axis='columns',inplace=True)
cocomo.set_axis(range(1,64),axis=0 ,inplace=True)
cocomo.rename_axis("Features", axis=1,inplace=True)
cocomo.rename_axis("Projects", axis=0,inplace=True)
dataset = cocomo

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Split the dataset into features (X) and target (y)
X = dataset.drop(columns=['effort'])
y = dataset['effort']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

pbounds = {'C': (0.1, 10),
           'epsilon': (0.01, 1),
           'gamma': (0.01, 100),
           'kernel': (0, 3)  # Use indices for categorical parameter
}
kernel_mapping = {0: 'linear', 1: 'rbf', 2: 'poly', 3: 'sigmoid'}

def svr_model(kernel,C, epsilon, gamma):
    model = SVR( kernel = kernel_mapping[int(kernel)], C=C, epsilon=epsilon, gamma=gamma)

    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.
    
    
# Perform Bayesian optimization.
optimizer = BayesianOptimization(
    f=svr_model,
    pbounds=pbounds,
    random_state=42,
    verbose=2,
)


optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

# Print the best hyperparameters found.
best_params = optimizer.max['params']
print("Best Hyperparameters:")
print(best_params)

best_C = best_params['C']
best_epsilon = best_params['epsilon']
best_gamma = best_params['gamma']
best_kernel= kernel_mapping[int(best_params['kernel'])]

# Train the final SVR model with the best parameters
final_model = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma,kernel= best_kernel)
final_model.fit(X_train_scaled, y_train)
    
# Make predictions on the test set
y_pred = final_model.predict(X_test_scaled)
    
# Calculate negative mean squared error (to maximize)
mse = -mean_squared_error(y_test, y_pred)
print(mse)

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Calculate Mean Magnitude of Relative Error (MMRE)
mmre = np.mean(np.abs((y_test - y_pred) / y_test))
print(f"Mean Magnitude of Relative Error (MMRE): {mmre:.2f}")

# Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")


|   iter    |  target   |     C     |  epsilon  |   gamma   |  kernel   |
-------------------------------------------------------------------------
| 1         | -593.7    | 3.808     | 0.9512    | 732.0     | 1.796     |
| 2         | -1.118e+0 | 1.645     | 0.1644    | 58.08     | 2.599     |
| 3         | -1.117e+0 | 6.051     | 0.711     | 20.59     | 2.91      |
| 4         | -515.3    | 8.341     | 0.2202    | 181.8     | 0.5502    |
| 5         | -549.4    | 3.112     | 0.5295    | 431.9     | 0.8737    |
| 6         | -593.6    | 6.157     | 0.1481    | 292.1     | 1.099     |
| 7         | -593.7    | 4.615     | 0.7873    | 199.7     | 1.543     |
| 8         | -532.4    | 5.965     | 0.05599   | 607.5     | 0.5116    |
| 9         | -1.117e+0 | 0.744     | 0.9494    | 965.6     | 2.425     |
| 10        | -593.7    | 3.116     | 0.1067    | 684.2     | 1.32      |
| 11        | -563.9    | 10.0      | 0.01      | 519.9     | 3.0       |
| 12        | -510.2    | 10.0      | 

In [1]:
import numpy as np
import random
import tensorflow as tf  # Import TensorFlow if you're using it
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

# Load your dataset and preprocess it
# Make sure you've properly loaded the dataset and assigned it to the 'dataset' variable

cocomo = pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomo81_dataset.csv", header=None)
columns_cocomo = ['rely', 'data', 'cplx', 'time', 'stor', 'virt', 'turn', 'acap', 'aexp', 'pcap', 'vexp', 'lexp', 'modp', 'tool', 'sced', 'loc', 'effort']
cocomo.set_axis(columns_cocomo, axis='columns', inplace=True)
cocomo.set_axis(range(1, 64), axis=0, inplace=True)
cocomo.rename_axis("Features", axis=1, inplace=True)
cocomo.rename_axis("Projects", axis=0, inplace=True)
dataset = cocomo

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Split the dataset into features (X) and target (y)
X = dataset.drop(columns=['effort'])
y = dataset['effort']

# Standardize the features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

pbounds = {'C': (0.1, 10),
           'epsilon': (0.01, 1),
           'gamma': (0.01, 100),
           'kernel': (0, 3)  # Use indices for categorical parameter
}
kernel_mapping = {0: 'linear', 1: 'rbf', 2: 'poly', 3: 'sigmoid'}


def svr_model(C, epsilon, gamma, kernel):
    model = SVR(kernel=kernel_mapping[int(kernel)], C=C, epsilon=epsilon, gamma=gamma)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

# Perform Bayesian optimization with k-fold cross-validation.
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
mean_RMSE=[]
mean_MMRE=[]
mean_MAE=[]

for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y.iloc[train_index], y.iloc[val_index]
    
    optimizer = BayesianOptimization(
        f=svr_model,
        pbounds=pbounds,
        random_state=42,
        verbose=0,
    )
    
    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.
    
    best_params = optimizer.max['params']
    best_C = best_params['C']
    best_epsilon = best_params['epsilon']
    best_gamma = best_params['gamma']
    best_kernel = kernel_mapping[int(best_params['kernel'])]
    
    final_model = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma, kernel=best_kernel)
    final_model.fit(X_train, y_train)
    
    y_pred = final_model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
    
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")   
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")


Mean Absolute Error mean: 605.6764682605556
Mean Magnitude of Relative Error mean (MMRE): 5.83
Root Mean Squared Error (RMSE) mean: 1140.2366792144612
